In [1]:
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import pandas as pd
import copy
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from skimage.transform import rotate


Using TensorFlow backend.


In [2]:
with open('train-1.renju') as f:
    content = f.readlines()
data = [x.strip() for x in content] 

' '.join(list(filter(lambda a: 'p' not in a, data[34])))

f = open('renju-mod', 'w')
for elem in data:
    list(filter(lambda a: 'p' not in a, elem))
    f.write(' '.join(list(filter(lambda a: 'p' not in a, elem))) +'\n')  # python will convert \n to os.linesep
f.close()

In [3]:
def get_number(letter):
    #lst = {'a':0, 'b':1,}

    return ord(letter) - ord('a')

def get_pos(elem):
    w = get_number(elem[0])
    h = int(elem[1:]) - 1
    return w, h

In [4]:
test_data = data[int(0.7 * len(data)):]

In [5]:
data = data[: int(0.7 * len(data))]

In [20]:
from itertools import islice
def data_gen(data, test=False):
    part_size = len(data)//410
    while(1):
        neural_games = []
        neural_ans = []
        while(len(neural_games) < part_size):
            elem = data[np.random.randint(len(data))]
            cur_pos = np.zeros((15,15, 3))
            game = elem.split()
            for iterator in range(1, len(game) - 1):
                elem = game[iterator]
                w, h = get_pos(elem)
                if iterator % 2 == 1:
                    cur_pos[w][h][0] = 1 
                else:
                    cur_pos[w][h][1] = 1 
                
                for i in range(15):
                    for j in range(15):
                        if cur_pos[i][j][0] > 0:
                            cur_pos[i][j][2] += 1
                        
                        
                cur_pos_to_app = copy.deepcopy(cur_pos)
                w, h = get_pos(game[iterator + 1])
                ans = h * 15 + w
                #print(cur_pos)
                neural_ans.append(ans)
                neural_games.append(cur_pos_to_app)
                if test == False:
                    neural_ans.append(ans)
                    neural_ans.append(ans)
                    neural_ans.append(ans)
                    neural_games.append(np.rot90(cur_pos_to_app, 1))
                    neural_games.append(np.rot90(cur_pos_to_app, 2))
                    neural_games.append(np.rot90(cur_pos_to_app, 3))
                if (len(neural_games) >= part_size):
                    break


        neural_games = np.array(neural_games)
        neural_ans = np.array(neural_ans)
        neural_ans = label_binarize(neural_ans, classes=[i for i in range(225)])
        neural_games.resize((part_size, 1, 15,15,3))
        yield (neural_games, neural_ans)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Convolution2D, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.layers import GRU, LSTM

model = Sequential()

model.add(Reshape((15,15,3), input_shape=(1,15,15,3)))
model.add(Convolution2D(2, (3,3), input_shape=(15, 15, 4), activation='relu'))
model.add(Convolution2D(4, (3,3), activation='relu'))
model.add(Convolution2D(6, (3,3), activation='relu'))
model.add(Convolution2D(8, (3,3), activation='relu'))
model.add(Convolution2D(12, (3,3), activation='relu'))
model.add(Convolution2D(16, (3,3), activation='relu'))


model.add(Flatten(name='flatten'))

model.add(Dense(3300, activation='relu', name='fczerominus2'))
model.add(BatchNormalization())
model.add(Dense(2200, activation='relu', name='fczeromins1'))
model.add(BatchNormalization())
model.add(Dense(1100, activation='relu', name='fczero'))
model.add(BatchNormalization())
model.add(Dense(550, activation='relu', name='fc1'))
model.add(BatchNormalization())
model.add(Dense(225, activation='softmax', name='fc2'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 15, 15, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 2)         56        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 4)         76        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 6)           222       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 8)           440       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 5, 5, 12)          876       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 16)          1744      
__________

In [7]:
from keras.callbacks import ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback, TQDMCallback
# keras, model definition...
model_saver = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.6f}.hdf5",
                                              monitor='val_acc', verbose=0,
                                              save_best_only=True, save_weights_only=False,
                                              mode='auto')
tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', 
                                          histogram_freq=0, 
                                          write_graph=True,
                                          write_images=True)

In [ ]:
model.fit_generator(data_gen(data),
        steps_per_epoch=500, epochs=350, verbose=0,
                   validation_data = data_gen(test_data, test=True), validation_steps=2,
                   callbacks=[model_saver, tensorboard, TQDMNotebookCallback()])

In [16]:
model.save("BigConv")

In [13]:
from keras.models import load_model
model_check_env = load_model("ReshapedModelsmall")

In [9]:
from keras.models import load_model, save_model

#save_model(model, "med_modelv1")

In [10]:
model = load_model("ReshapedModel0.46-9")

In [15]:
model_policy = load_model("ReshapedModel0.46-9")

In [16]:
model = load_model("ReshapedModelsmall")

In [12]:
class Renju(object): 
    def __init__(self, player):
        """
        player : 1 for black, 2 for white
        """
        self.cur_pos = np.zeros((15,15, 3))
        self.cur_player = player
        self.player = player
        self.action_space = 225
        
        
    def in_step(self, action):
        """
        Run one timestep of the environment's dynamics. When end of episode
        is reached, reset() should be called to reset the environment's internal state.
        Input
        -----
        action : an action provided by the environment
        Outputs
        -------
        (observation, reward, done, info)
        observation : agent's observation of the current environment
        reward [Float] : amount of reward due to the previous action
        done : a boolean, indicating whether the episode has ended
        info : a dictionary containing other diagnostic information from the previous action
        """
        if self.cur_player == 1:
            self.cur_pos[action % 15][action // 15][0] = 1
        else:
            self.cur_pos[action % 15][action // 15][1] = 1

        
        if self.cur_player == 1:
            self.cur_player = 2
        else:
            self.cur_player = 1
        
        
        for i in range(15):
            for j in range(15):
                if self.cur_pos[i][j][0] > 0:
                    self.cur_pos[i][j][2] += 1
                
        reward = 0
        
        def check_limit(i, j):
            if i >= 0 and i <= 14 and j >= 0 and j <= 14:
                return True
            else:
                return False
        
        for i in range(15):
            for j in range(15):
                if check_limit(i + 5, j):
                    if self.cur_pos[i][j][0] == self.cur_pos[i + 1][j][0] == self.cur_pos[i + 2][j][0] == self.cur_pos[i + 3][j][0] == self.cur_pos[i + 4][j][0] == self.cur_pos[i + 5][j][0]:
                        if self.cur_pos[i][j][0] == 1:
                            if self.player == 1:
                                reward = 1
                            else:
                                reward = -1
                            
                if check_limit(i, j + 5):
                    if self.cur_pos[i][j][0] == self.cur_pos[i][j + 1][0] == self.cur_pos[i][j + 2][0] == self.cur_pos[i][j + 3][0] == self.cur_pos[i][j + 4][0] == self.cur_pos[i][j + 5][0]:
                        if self.cur_pos[i][j][0] == 1:
                            if self.player == 1:
                                reward = 1
                            else:
                                reward = -1
                            
                if check_limit(i - 5, j):
                    if self.cur_pos[i][j][0] == self.cur_pos[i - 1][j][0] == self.cur_pos[i - 2][j][0] == self.cur_pos[i - 3][j][0] == self.cur_pos[i - 4][j][0] == self.cur_pos[i - 5][j][0]:
                        if self.cur_pos[i][j][0] == 1:
                            if self.player == 1:
                                reward = 1
                            else:
                                reward = -1
                
                
                
                if check_limit(i, j - 5):
                    if self.cur_pos[i][j][0] == self.cur_pos[i][j - 1][0] == self.cur_pos[i][j - 2][0] == self.cur_pos[i][j - 3][0] == self.cur_pos[i][j - 4][0] == self.cur_pos[i][j - 5][0]:
                        if self.cur_pos[i][j][0] == 1:
                            if self.player == 1:
                                reward = 1
                            else:
                                reward = -1
                
                
                
                if check_limit(i + 5, j + 5):
                    if self.cur_pos[i][j][0] == self.cur_pos[i + 1][j + 1][0] == self.cur_pos[i + 2][j + 2][0] == self.cur_pos[i + 3][j + 3][0] == self.cur_pos[i + 4][j + 4][0] == self.cur_pos[i + 5][j + 5][0]:
                        if self.cur_pos[i][j][0] == 1:
                            if self.player == 1:
                                reward = 1
                            else:
                                reward = -1
                            
                if check_limit(i - 5, j - 5):
                    if self.cur_pos[i][j][0] == self.cur_pos[i - 1][j - 1][0] == self.cur_pos[i - 2][j - 2][0] == self.cur_pos[i - 3][j - 3][0] == self.cur_pos[i - 4][j - 4][0] == self.cur_pos[i - 5][j - 5][0]:
                        if self.cur_pos[i][j][0] == 1:
                            if self.player == 1:
                                reward = 1
                            else:
                                reward = -1
                
        
        done = True if reward != 0 else False
        cur_pos = self.cur_pos
        if done:
            self.reset()
        info = dict()
        return (cur_pos, reward, done, info)
    
    def step(self, action):
        cur_pos, reward, done, info = self.in_step(action)
        if reward != 0:
            return cur_pos, reward, done, info
        else:
            return self.in_step(np.argmax(model_check_env.predict(np.array([[env.cur_pos]]))))
            

    def reset(self):
        """
        Resets the state of the environment, returning an initial observation.
        Outputs
        -------
        observation : the initial observation of the space. (Initial reward is assumed to be 0.)
        """
        self.cur_pos = np.zeros((15,15,3))
        self.cur_player = 1
        return self.cur_pos
    

In [13]:
class RenjuTEST(object): 
    def __init__(self, player, RL):
        """
        player : 1 for black, 2 for white
        """
        self.cur_pos = np.zeros((15,15, 3))
        self.cur_player = player
        self.player = player
        self.action_space = 225
        self.RL = RL
        self.moves_done = 0
        self.ext_pos = np.zeros((25, 25))
        
        
    def in_step(self, action):
        """
        Run one timestep of the environment's dynamics. When end of episode
        is reached, reset() should be called to reset the environment's internal state.
        Input
        -----
        action : an action provided by the environment
        Outputs
        -------
        (observation, reward, done, info)
        observation : agent's observation of the current environment
        reward [Float] : amount of reward due to the previous action
        done : a boolean, indicating whether the episode has ended
        info : a dictionary containing other diagnostic information from the previous action
        """
        self.moves_done += 1
        if self.cur_player == 1:
            self.cur_pos[action % 15][action // 15][0] = 1
        else:
            self.cur_pos[action % 15][action // 15][1] = 1
            
            
        w = (action % 15) + 5
        h = (action // 15) + 5
        
        self.ext_pos[w][h] = self.cur_player
        
        
        
        
        for i in range(15):
            for j in range(15):
                if self.cur_pos[i][j][0] > 0:
                    self.cur_pos[i][j][2] += 1
                
        reward = 0
        
        rdiag = [0, 0, 0]
        rdiaginv = [0, 0, 0]
        ldiag = [0, 0, 0]
        ldiaginv = [0, 0, 0]
        rrow = [0, 0, 0]
        lrow = [0, 0, 0]
        rcol = [0, 0, 0]
        lcol = [0, 0, 0]
        
        broken = [0,0,0,0,0,0,0,0]
        
        for i in range(5):
            
            if self.ext_pos[w + i][h + i]:
                if not broken[0]:
                    rdiag[int(self.ext_pos[w + i][h + i])] += 1
                else:
                    broken[0] = 1
                
            if self.ext_pos[w][h + i]:
                if not broken[1]:
                    rcol[int(self.ext_pos[w][h + i])] += 1
                else:
                    broken[1] = 1
                
            if self.ext_pos[w - i][h + i]:
                if not broken[2]:
                    ldiag[int(self.ext_pos[w - i][h + i])] += 1
                else:
                    broken[2] = 1
                
            
            if self.ext_pos[w - i][h]:
                if not broken[3]:
                    lrow[int(self.ext_pos[w - i][h])] += 1
                else:
                    broken[3] = 1
            
            if self.ext_pos[w - i][h - i]:
                if not broken[4]:
                    rdiaginv[int(self.ext_pos[w - i][h - i])] += 1
                else:
                    broken[4] = 1
            
            if self.ext_pos[w][h - i]:
                if not broken[5]:
                    lcol[int(self.ext_pos[w][h - i])] += 1
                else:
                    broken[5] = 1
            
            if self.ext_pos[w + i][h - i]:
                if not broken[6]:
                    ldiaginv[int(self.ext_pos[w + i][h - i])] += 1
                else:
                    broken[6] = 1
            
            if self.ext_pos[w + i][h]:
                if not broken[7]:
                    rrow[int(self.ext_pos[w + i][h])] += 1
                else:
                    broken[7] = 1
        
        
        if rdiag[self.cur_player] + rdiaginv[self.cur_player] >= 6:
            if self.player == self.cur_player:
                reward = 1
            else:
                reward = -1
                
        if ldiag[self.cur_player] + ldiaginv[self.cur_player] >= 6:
            if self.player == self.cur_player:
                reward = 1
            else:
                reward = -1
        
        if rrow[self.cur_player] + lrow[self.cur_player] >= 6:
            if self.player == self.cur_player:
                reward = 1
            else:
                reward = -1
                
        if lcol[self.cur_player] + rcol[self.cur_player] >= 6:
            if self.player == self.cur_player:
                reward = 1
            else:
                reward = -1
        """
        print('rdiag', rdiag)
        print('rdiaginv', rdiaginv)
        print('ldiag', ldiag)
        print('ldiaginv', ldiaginv)
        print('rrow', rrow)
        print('lrow', lrow)
        print('rcol', rcol)
        print('lcol', lcol)
        """
        
        if self.cur_player == 1:
            self.cur_player = 2
        else:
            self.cur_player = 1
        
        done = True if (self.moves_done == 225 or reward != 0) else False
        cur_pos = self.cur_pos
        if self.moves_done == 225:
            self.reset()
        info = dict()
        return (cur_pos, reward, done, info)
    
    def net_ans(self, model, mode = 'all'):
        s = model.predict(np.array([[self.cur_pos]]))[0]
        if mode == 'one':
            return np.argmax(s)
        else:
            #return sorted(range(len(s)), key=lambda k: s[k], reverse=True)
            return np.argsort(s)
    
    def step(self, action):
        """ 
        RL_move = dqn.compute_q_values([env.cur_pos])
        RL_move = sorted(range(len(RL_move)), key=lambda k: RL_move[k], reverse=True)
        
        action = 0
        for act in RL_move:
            if self.cur_pos[act % 15][act // 15][0] == 0 and self.cur_pos[act % 15][act // 15][1] == 0:
                action = act
                break

        
        print("RL:", action)
        
        net_move = self.net_ans(model)
            
        action = 0
        for act in net_move:
            if self.cur_pos[act % 15][act // 15][0] == 0 and self.cur_pos[act % 15][act // 15][1] == 0:
                action = act
                break
       
         """
        cur_pos, reward, done, info = self.in_step(action)
        
        if done:
            self.reset()
        #print("Net:", action)
        
        """
        for i in range(15):
            for j in range(15):
                print(self.cur_pos[i][j][1], end=' ')
            print("\n")
        """
            
        if reward != 0:
            return cur_pos, reward, done, info
        else:
            s = model.predict(np.array([[self.cur_pos]]))[0]
            action = np.argmax(s)
            if self.cur_pos[action % 15][action // 15][0] != 0 or self.cur_pos[action % 15][action // 15][1] != 0:
                net_move = np.argsort(s)[::-1]
            
                action = 0
                for act in net_move:
                    if self.cur_pos[act % 15][act // 15][0] == 0 and self.cur_pos[act % 15][act // 15][1] == 0:
                        action = act
                        break
                #print("Net:", action)
            return self.in_step(action)
            
    
    def render(self, mode='human'):
        if mode == 'human':
            for i in range(15):
                for j in range(15):
                    flag = 0
                    if self.cur_pos[i][j][0] == 1:
                        flag = 1
                        print("X", end=' ')
                    if self.cur_pos[i][j][1] == 1:
                        flag = 1
                        print("O", end=' ')
                    if not flag:
                        print("_", end=' ')
                print('\n', end='')
            print("------------------------------------------------\n")
        if mode == 'debug':
            for i in range(25):
                for j in range(25):
                    flag = 0
                    if self.ext_pos[i][j] == 1:
                        flag = 1
                        print("X", end=' ')
                    if self.ext_pos[i][j] == 2:
                        flag = 1
                        print("O", end=' ')
                    if not flag:
                        print("_", end=' ')
                print('\n', end='')
            print("------------------------------------------------\n")
        
        
    def reset(self):
        """
        Resets the state of the environment, returning an initial observation.
        Outputs
        -------
        observation : the initial observation of the space. (Initial reward is assumed to be 0.)
        """
        self.cur_pos = np.zeros((15,15,3))
        self.cur_player = 1
        self.moves_done = 0
        self.ext_pos = np.zeros((25, 25))
        return self.cur_pos
    

In [51]:
from tqdm import tqdm

tester = RenjuTEST(1, None)

win = 0
loose = 0
all_game = 0


for i in tqdm(range(2000)):
    cur_pos, reward, done, info = tester.step(1)
    if reward == 1:
        win += 1
    if reward == -1:
        loose += 1
    if done:
        all_game += 1
    #tester.render(mode='human')

100%|██████████| 2000/2000 [00:13<00:00, 153.70it/s]


In [52]:
%lprun -f  tester.step tester.step(1)

In [32]:
indices

(array([0]), array([3]))

In [40]:
print("win:", win, "loose:", loose, "all rate:", win / all_game)

win: 222 loose: 0 all rate: 1.0


In [22]:
np.argmax(model_check_env.predict(np.array([[env.cur_pos]])))

205

In [55]:
def net_ans(env):
    s = model_check_env.predict(np.array([[env.cur_pos]]))[0]
    return sorted(range(len(s)), key=lambda k: s[k], reverse=True)

net_ans(env)

[205,
 157,
 79,
 191,
 63,
 202,
 22,
 175,
 111,
 91,
 189,
 152,
 142,
 34,
 25,
 160,
 72,
 33,
 204,
 36,
 176,
 19,
 57,
 48,
 137,
 178,
 133,
 52,
 136,
 106,
 151,
 132,
 77,
 187,
 102,
 103,
 92,
 26,
 118,
 88,
 20,
 42,
 109,
 35,
 67,
 172,
 121,
 126,
 78,
 93,
 119,
 169,
 61,
 39,
 177,
 153,
 71,
 24,
 148,
 59,
 201,
 18,
 162,
 107,
 161,
 104,
 139,
 122,
 146,
 64,
 62,
 190,
 127,
 186,
 194,
 199,
 41,
 167,
 198,
 74,
 166,
 56,
 124,
 40,
 215,
 131,
 134,
 174,
 159,
 164,
 51,
 182,
 87,
 117,
 37,
 140,
 110,
 155,
 185,
 168,
 76,
 54,
 171,
 154,
 7,
 206,
 49,
 193,
 149,
 73,
 90,
 123,
 163,
 31,
 147,
 183,
 192,
 138,
 108,
 58,
 89,
 4,
 170,
 65,
 69,
 116,
 70,
 94,
 145,
 66,
 47,
 50,
 184,
 200,
 44,
 21,
 5,
 2,
 27,
 86,
 125,
 80,
 105,
 97,
 101,
 209,
 141,
 156,
 220,
 179,
 181,
 55,
 17,
 75,
 208,
 43,
 219,
 216,
 30,
 14,
 96,
 6,
 223,
 46,
 120,
 150,
 32,
 12,
 213,
 135,
 212,
 217,
 95,
 195,
 222,
 15,
 3,
 196,
 10,
 214,
 13,

In [6]:
for i in range(len(model_normal.layers)):
    model.layers[i + 1].set_weights(model_normal.layers[i].get_weights())

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Convolution2D, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.layers import GRU, LSTM

model_policy = Sequential()

model_policy.add(Reshape((15,15,3), input_shape=(1,15,15,3)))
model_policy.add(Convolution2D(8, (3,3), input_shape=(15, 15, 4), activation='relu'))

model_policy.add(Flatten(name='flatten'))


model_policy.add(Dense(550, activation='relu', name='fc1'))

model_policy.add(Dense(225, activation='softmax', name='fc2'))
model_policy.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_policy.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 15, 15, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 8)         224       
_________________________________________________________________
flatten (Flatten)            (None, 1352)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 550)               744150    
_________________________________________________________________
fc2 (Dense)                  (None, 225)               123975    
Total params: 868,349.0
Trainable params: 868,349.0
Non-trainable params: 0.0
_________________________________________________________________


In [18]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
from rl.agents.sarsa import SarsaAgent


ENV_NAME = 'Renju'


# Get the environment and extract the number of actions.
env = RenjuTEST(1, None)
np.random.seed(123)
nb_actions = env.action_space


policy = BoltzmannQPolicy()
sarsa = SarsaAgent(model=model_policy, nb_actions=nb_actions, nb_steps_warmup=10, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
sarsa.fit(env, nb_steps=5000, visualize=False, verbose=2)

# After training is done, we save the final weights.
sarsa.save_weights('sarsa_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=False)

Training for 5000 steps ...
   12/5000: episode: 1, duration: 1.496s, episode steps: 12, steps per second: 8, episode reward: -1.000, mean reward: -0.083 [-1.000, 0.000], mean action: 121.833 [51.000, 220.000], mean observation: 0.109 [0.000, 24.000], loss: 0.502264, mean_absolute_error: 0.008889, mean_q: 0.023879
   24/5000: episode: 2, duration: 0.256s, episode steps: 12, steps per second: 47, episode reward: -1.000, mean reward: -0.083 [-1.000, 0.000], mean action: 115.083 [13.000, 191.000], mean observation: 0.103 [0.000, 24.000], loss: 0.000005, mean_absolute_error: 0.004432, mean_q: 0.009392
   31/5000: episode: 3, duration: 0.138s, episode steps: 7, steps per second: 51, episode reward: 1.000, mean reward: 0.143 [0.000, 1.000], mean action: 78.857 [20.000, 141.000], mean observation: 0.046 [0.000, 13.000], loss: 0.000003, mean_absolute_error: 0.004433, mean_q: 0.007376
   39/5000: episode: 4, duration: 0.172s, episode steps: 8, steps per second: 46, episode reward: -1.000, mean 

In [23]:
for epoch in range(10):
    sarsa.fit(env, nb_steps=5000, visualize=False, verbose=0)

    # After training is done, we save the final weights.
    sarsa.save_weights('sarsa_{}_weights.h5f'.format(ENV_NAME + str(epoch)), overwrite=True)

    # Finally, evaluate our algorithm for 5 episodes.
    sarsa.test(env, nb_episodes=5, visualize=False)

Testing for 5 episodes ...
Episode 1: reward: -1.000, steps: 5
Episode 2: reward: -1.000, steps: 16
Episode 3: reward: -1.000, steps: 7
Episode 4: reward: -1.000, steps: 7
Episode 5: reward: -1.000, steps: 8
Testing for 5 episodes ...
Episode 1: reward: -1.000, steps: 10
Episode 2: reward: -1.000, steps: 19
Episode 3: reward: -1.000, steps: 5
Episode 4: reward: -1.000, steps: 7
Episode 5: reward: -1.000, steps: 10
Testing for 5 episodes ...
Episode 1: reward: -1.000, steps: 9
Episode 2: reward: -1.000, steps: 13
Episode 3: reward: -1.000, steps: 7
Episode 4: reward: -1.000, steps: 6
Episode 5: reward: -1.000, steps: 9
Testing for 5 episodes ...
Episode 1: reward: -1.000, steps: 10
Episode 2: reward: -1.000, steps: 11
Episode 3: reward: -1.000, steps: 21
Episode 4: reward: 1.000, steps: 17
Episode 5: reward: -1.000, steps: 11
Testing for 5 episodes ...
Episode 1: reward: -1.000, steps: 20
Episode 2: reward: -1.000, steps: 7
Episode 3: reward: -1.000, steps: 16
Episode 4: reward: -1.000,

In [47]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [23]:
%lprun -f dqn.fit dqn.fit(env, nb_steps=17500, log_interval=10000, visualize=False, verbose=2)

Training for 17500 steps ...
     8/17500: episode: 1, duration: 0.174s, episode steps: 8, steps per second: 46, episode reward: -1.000, mean reward: -0.125 [-1.000, 0.000], mean action: 113.375 [6.000, 224.000], mean observation: 0.058 [0.000, 16.000], loss: --, acc: --, mean_q: --, mean_eps: --
    14/17500: episode: 2, duration: 0.092s, episode steps: 6, steps per second: 65, episode reward: -1.000, mean reward: -0.167 [-1.000, 0.000], mean action: 67.167 [1.000, 140.000], mean observation: 0.038 [0.000, 12.000], loss: --, acc: --, mean_q: --, mean_eps: --
    21/17500: episode: 3, duration: 0.104s, episode steps: 7, steps per second: 67, episode reward: -1.000, mean reward: -0.143 [-1.000, 0.000], mean action: 122.000 [45.000, 194.000], mean observation: 0.047 [0.000, 14.000], loss: --, acc: --, mean_q: --, mean_eps: --
    29/17500: episode: 4, duration: 0.124s, episode steps: 8, steps per second: 65, episode reward: -1.000, mean reward: -0.125 [-1.000, 0.000], mean action: 120.50

In [69]:
env = RenjuTEST(2, None)
dqn.test(env, nb_episodes=2, visualize=True)

Testing for 2 episodes ...
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ O _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ X _ _ _ _ _ _ _ 

------------------------------------------------
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ O O _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

_ _ _ _ _ _ _ _ _ _ _ _ _ _ 

In [47]:
loose

0

In [73]:
dqn.compute_q_values([env.cur_pos])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0